In [1]:
import os
import json
from typing import List, Dict
from pathlib import Path

import numpy as np
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langdetect import detect, DetectorFactory

In [2]:
DetectorFactory.seed = 0 #fix langdetect randomness

In [3]:
# base paths
BASE_DIR = Path().resolve()
DOCS_DIR = BASE_DIR / "data" / "docs"
INDEX_DIR = BASE_DIR / "index"

DOCS_DIR, INDEX_DIR

(PosixPath('/Users/arturschaefer/Documents/Freelancing/Repos/rag-llm-example/data/docs'),
 PosixPath('/Users/arturschaefer/Documents/Freelancing/Repos/rag-llm-example/index'))

In [4]:
# load markdown documents

def load_documents(docs_dir: Path) -> List[Dict]:
    documents = []
    for path in sorted(docs_dir.glob("*.md")):
        text = path.read_text(encoding="utf-8")
        lines = text.splitlines()
        
        # use first Markdown heading as title
        if lines and lines[0].lstrip().startswith("#"):
            title = lines[0].lstrip("#").strip()
        else:
            title = path.stem
        
        documents.append({
            "id": path.stem,
            "title": title,
            "text": text
        })
    return documents

docs = load_documents(DOCS_DIR)
len(docs), [d["id"] for d in docs]


(5,
 ['employee_handbook',
  'it_security',
  'remote_work_policy',
  'travel_expenses',
  'vacation_policy'])

In [5]:
# chunk documents (simple double-newline split)

def chunk_document(doc: Dict, min_chars: int = 40) -> List[Dict]:
    raw_chunks = [c.strip() for c in doc["text"].split("\n\n") if c.strip()]
    chunks = []

    for i, c in enumerate(raw_chunks):
        if len(c) < min_chars:
            continue
        chunks.append({
            "chunk_id": f"{doc['id']}-{i}",
            "doc_id": doc["id"],
            "title": doc["title"],
            "text": c,
        })
    return chunks

all_chunks = []
for d in docs:
    all_chunks.extend(chunk_document(d))

len(all_chunks)


20

In [6]:
# create embeddings

EMBED_MODEL = "sentence-transformers/distiluse-base-multilingual-cased-v2"
embed_model = SentenceTransformer(EMBED_MODEL)

texts = [c["text"] for c in all_chunks]
embeddings = embed_model.encode(texts, convert_to_numpy=True, normalize_embeddings=True)


In [7]:
# save embeddings

np.save(INDEX_DIR / "embeddings.npy", embeddings)
with open(INDEX_DIR / "chunks.json", "w", encoding="utf-8") as f:
    json.dump(all_chunks, f, ensure_ascii=False, indent=2)


In [8]:
# retriever (semantic search)

def retrieve(query: str, k: int = 5) -> List[Dict]:
    q_emb = embed_model.encode([query], normalize_embeddings=True)[0]
    sims = embeddings @ q_emb  # cosine similarity since normalized
    top_idx = np.argsort(-sims)[:k]

    results = []
    for idx in top_idx:
        c = all_chunks[idx]
        results.append({
            "chunk_id": c["chunk_id"],
            "doc_id": c["doc_id"],
            "title": c["title"],
            "text": c["text"],
            "score": float(sims[idx]),
        })
    return results

retrieve("Wie viele Urlaubstage habe ich?", k=3)


[{'chunk_id': 'vacation_policy-1',
  'doc_id': 'vacation_policy',
  'title': 'Vacation Policy',
  'text': '**1. Annual Leave Entitlement**  \nEmployees receive 25 days of paid vacation per year. Leave is accrued monthly during the first year.',
  'score': 0.31855595111846924},
 {'chunk_id': 'vacation_policy-2',
  'doc_id': 'vacation_policy',
  'title': 'Vacation Policy',
  'text': '**2. Request Procedure**  \nVacation requests must be submitted via the HR portal at least two weeks in advance. Approval is subject to team workload and staffing requirements.',
  'score': 0.259296715259552},
 {'chunk_id': 'remote_work_policy-1',
  'doc_id': 'remote_work_policy',
  'title': 'Remote Work Policy',
  'text': '**1. Eligibility**  \nEmployees may work remotely up to three days per week, subject to manager approval.',
  'score': 0.2439272403717041}]

In [9]:
# language detection

def detect_language(text: str) -> str:
    cleaned = text.strip()
    if not cleaned:
        return "en"
     
    lang = detect(cleaned)
    if lang in ["de", "en"]:
        return lang
    else: return "en"

In [ ]:
# build the rag prompt - optimized for flan-t5

def build_prompt(query: str, contexts: List[Dict]) -> str:
    user_lang = detect_language(query)

    # keep context short – flan-t5-base has limited context
    context_parts = []
    for c in contexts:
        context_parts.append(c["text"])
    context_str = "\n\n".join(context_parts)

    if user_lang == "de":
        # Simplified instruction that works better with flan-t5
        instruction = (
            "Answer the question based on the context below. Answer in full sentences. "
            "Answer in German.\n\n"
        )
        prompt = (
            instruction +
            "Context:\n" + context_str + "\n\n" +
            "Frage: " + query + "\n" +
            "Antwort:"
        )
    else:
        instruction = (
            "Answer the question based on the context below. Answer in full sentences.\n\n"
        )
        prompt = (
            instruction +
            "Context:\n" + context_str + "\n\n" +
            "Question: " + query + "\n" +
            "Answer:"
        )

    return prompt


In [24]:
# local llm using transformers - Qwen model

model_name = "Qwen/Qwen2.5-3B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    torch_dtype="auto"
)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some parameters are on the meta device because they were offloaded to the disk.


In [25]:
# llm call helper - optimized for better context usage

def call_local_llm(messages: List[Dict], max_new_tokens: int = 200) -> str:
    # Apply chat template
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    
    # Tokenize
    model_inputs = tokenizer([text], return_tensors="pt").to(model.device)
    
    # Generate with parameters encouraging fuller responses
    generated_ids = model.generate(
        **model_inputs,
        max_new_tokens=max_new_tokens,
        do_sample=False,
        temperature=0.1,  # Slightly higher for more creative synthesis
        repetition_penalty=1.15,
        no_repeat_ngram_size=3  # Prevent repetitive phrases
    )
    
    # Decode only the new tokens (exclude input)
    generated_ids = [
        output_ids[len(input_ids):] 
        for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]
    
    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
    return response.strip()


In [26]:
# NOTE: Qwen2.5-3B-Instruct is a strong multilingual model
# It handles German much better than FLAN-T5
# Other good alternatives:
# - "Qwen/Qwen2.5-7B-Instruct" (larger, better quality)
# - "meta-llama/Llama-3.2-3B-Instruct" (similar size, good multilingual)


In [27]:
# full rag pipeline - optimized for comprehensive answers

def rag_answer(query: str, k: int = 3) -> Dict:
    # Retrieve relevant contexts
    contexts = retrieve(query, k=k)
    
    # Build chat messages with numbered sources
    messages = build_prompt(query, contexts)
    
    # Generate answer with more tokens for complete responses
    answer = call_local_llm(messages, max_new_tokens=200)

    return {
        "answer": answer,
        "sources": contexts,
        "num_sources": len(contexts)
    }


In [28]:
# Helper to see what context the model receives

def preview_prompt(query: str, k: int = 3):
    contexts = retrieve(query, k=k)
    messages = build_prompt(query, contexts)
    
    print("=== SYSTEM MESSAGE ===")
    print(messages[0]["content"])
    print("\n=== USER MESSAGE (first 800 chars) ===")
    print(messages[1]["content"][:800])
    print("...")
    print(f"\n[Total context length: {len(messages[1]['content'])} characters]")


In [31]:
res = rag_answer("How much vacation do I get?", k=3)

print("Antwort:")
print(res["answer"])

print("\n\nQuellen:")
for s in res["sources"]:
    print(f"- {s['title']}: {s['text'][:100]}... (score: {s['score']:.3f})")


Antwort:
Yes, I'm Qwen. How can I assist you today? Whether it's answering questions, providing information on various topics, or engaging in conversation, feel free to ask me anything!


Quellen:
- Vacation Policy: **1. Annual Leave Entitlement**  
Employees receive 25 days of paid vacation per year. Leave is accr... (score: 0.304)
- Vacation Policy: **2. Request Procedure**  
Vacation requests must be submitted via the HR portal at least two weeks ... (score: 0.215)
- Travel & Expense Guidelines: **3. Meals**  
Per-diem allowances follow local regulations. Receipts must be uploaded within ten da... (score: 0.209)
